In [1]:
import pandas as pd
from matplotlib import pyplot as plt
df = pd.read_csv("churn-5000.csv")

In [2]:
num_df = df.select_dtypes(include=['float64', 'int64']).copy()
cat_df = df.select_dtypes(include=['object']).copy()

# Categorical boolean mask
categorical_feature_mask = cat_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_cols = cat_df.columns[categorical_feature_mask].tolist()

import numpy as np
#conData=np.log(0.00001 + 1)
conData=0
cat_df=cat_df.fillna(conData)
num_df=num_df.fillna(conData)
cat_df=cat_df.astype(str)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
cat_df[categorical_cols] = cat_df[categorical_cols].apply(lambda col: le.fit_transform(col))

#cat_df[categorical_cols].head(10)


churn=cat_df['churn']
cat_df=cat_df.drop(['churn'], axis=1)



num_df=num_df.fillna(conData)

num_df=num_df.replace([np.inf, -np.inf], np.nan)
num_df=num_df.fillna(conData)
num_df[num_df < 0] = 0
#num_df.to_csv('D://num_df.csv')

In [3]:
for col in num_df.columns:
    Bins=20
    Omega=num_df[col].max()-num_df[col].min()
    #Omega=num_df[col].max()-0
    Omega=Omega/Bins
    #print(num_df[col].min())
    #BINValue=[1.1, 1.1]
    BINValue=[]
    BINValue.insert(0,num_df[col].min())
    #BINValue.insert(0,-1)
    for i in range(19):
        BINValue.insert(i+1,BINValue[i]+Omega)
        #print(BINValue[i+1])
    #print(BINValue)
    #print(num_df[col])
    labels =[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
    cats=pd.cut(num_df[col], BINValue, labels=labels)
    num_df[col]=cats
    #print(cats)
    #cats.to_csv('D://cats.csv')
    #break

In [4]:
num_df=num_df.fillna(1)
result_df = pd.concat([num_df, cat_df], axis=1)
np.nan_to_num(result_df)

#result_df=result_df.replace([np.inf, -np.inf], np.nan)
#result_df=result_df.fillna(-1)
#result_df=result_df.abs()

result_df_op=result_df
#result_df_op=result_df_op.drop(['churn'], axis=1)
#result_df_op=np.nan_to_num(result_df_op)

X=result_df_op
#y=result_df['churn'] 
y=churn

In [5]:
X.shape

(5000, 19)

In [6]:
X_train=X
y_train=y
#Univariate Selection
from sklearn.feature_selection import SelectKBest, chi2
select_feature = SelectKBest(chi2, k=15).fit(X_train, y_train)
selected_features_df = pd.DataFrame({'Feature':list(X_train.columns),
                                     'Scores':select_feature.scores_})
selected_features_df.sort_values(by='Scores', ascending=False)

,Feature,Scores
15,number_customer_service_calls,398.959315
17,international_plan,303.963357
2,number_vmail_messages,295.325026
5,total_day_charge,128.245432
3,total_day_minutes,127.216812
18,voice_mail_plan,45.058340
12,total_intl_minutes,28.001521
6,total_eve_minutes,25.102043
13,total_intl_calls,17.874162
8,total_eve_charge,14.298142


In [7]:
y_train=y
x_train_chi = select_feature.transform(X_train)
x_train_chi.shape

(5000, 15)

In [8]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB()

classifier.fit(X,y_train)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

3985 308 417 290
pod:  0.4101838755304102
pof:  0.07174470067551829
AUC:  0.6692195874274459


In [9]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

classifier=GaussianNB();
#params = {
#          "priors" : "None",
#          "var_smoothing" : 1e-9
#}
#create an list of var_smoothing to cross validate
#steps = [1e-8, 1e-7, 1e-6, 1e-5, 1e-4]

params_NB = {'var_smoothing': np.logspace(0,-9, num=100)}

grid = GridSearchCV(estimator=classifier, 
                     param_grid=params_NB, 
                     cv=cv_method,
                     verbose=1, 
                     scoring='accuracy', n_jobs=-1)

grid.fit(x_train_chi, y_train)

 
print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 15 folds for each of 100 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.2s


{'var_smoothing': 0.0006579332246575676}
GaussianNB(priors=None, var_smoothing=0.0006579332246575676)
0.8750666666666667


[Parallel(n_jobs=-1)]: Done 1500 out of 1500 | elapsed:   11.1s finished


In [10]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB
#Create a Gaussian Classifier
classifier = GaussianNB(priors=None, var_smoothing=0.0006579332246575676)

classifier.fit(x_train_chi,y_train)


cv_method = RepeatedKFold(n_splits=5, 
                          n_repeats=3, 
                          random_state=999)

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 5)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

4196 97 522 185
pod:  0.26166902404526166
pof:  0.02259492196599115
AUC:  0.6195370510396353
accuracy:  0.8762


In [1]:
 #accuracy=(tp+tn)/(tp+fn+fp+tn)
accuracy=(26836+27976)/(27976+22462+22726+26836)
print ('accuracy: ',accuracy)

accuracy:  0.54812


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Specify parameters
c_values = list(np.arange(1, 10))


param_grid = [
    {'C': c_values, 'penalty': ['l1'], 'solver' : ['liblinear'], 'multi_class' : ['ovr']},    
    {'C': c_values, 'penalty': ['l2'], 'solver' : ['liblinear', 'newton-cg', 'lbfgs'], 'multi_class' : ['ovr']}
]
 

grid = GridSearchCV(LogisticRegression(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 36 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    3.3s


{'C': 1, 'multi_class': 'ovr', 'penalty': 'l2', 'solver': 'liblinear'}
LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
0.8628


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:    6.1s finished


In [12]:
#LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)
classifier.fit(x_train_chi, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4203 90 596 111
pod:  0.157001414427157
pof:  0.020964360587002098
AUC:  0.5680185269200775
accuracy:  0.8628


In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier  

# Specify parameters
c_values = list(np.arange(1, 10))
param_grid ={
   'n_neighbors': [3,5,11,19],
   'weights': ['uniform', 'distance'],
   'metric': ['euclidean', 'manhattan']
}

grid = GridSearchCV(KNeighborsClassifier(), param_grid, verbose=1, cv=3, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)



Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


{'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'uniform'}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')
0.8706


[Parallel(n_jobs=-1)]: Done  48 out of  48 | elapsed:   11.4s finished


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn import svm

# Specify parameters

Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
gammas = [0.001, 0.01, 0.1, 1, 1e-3, 1e-4]
param_grid = {'C': Cs, 'gamma' : gammas}

tuned_parameters = [{'kernel': ['rbf', 'linear'], 
                     'gamma': [0.001, 0.01, 0.1, 1, 1e-3, 1e-4], 
                     'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]} ]

grid = GridSearchCV(svm.SVC(), param_grid, verbose=1, cv=10, n_jobs=-1)
grid.fit(x_train_chi, y_train)

print(grid.best_params_)
print(grid.best_estimator_)
print(grid.best_score_)

Fitting 10 folds for each of 42 candidates, totalling 420 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 410.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 1993.6min


In [14]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform' )  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)
accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4258 35 608 99
pod:  0.14002828854314003
pof:  0.00815280689494526
AUC:  0.5659377408240974
accuracy:  0.8714


In [15]:
# KNN
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_train, y_train) 

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)


4255 38 627 80
pod:  0.11315417256011315
pof:  0.008851618914511996
AUC:  0.5521512768228005
accuracy:  0.867


In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}


#clf = RandomForestClassifier()

# Choose some parameter combinations to try
#parameters = {'n_estimators': [4, 6, 9], 
#              'max_features': ['log2', 'sqrt','auto'], 
#              'criterion': ['entropy', 'gini'],
#              'max_depth': [2, 3, 5, 10], 
#              'min_samples_split': [2, 3, 5],
#              'min_samples_leaf': [1,5,8]
#             }

#model_params = {
#    'n_estimators': [50, 150, 250],
#    'max_features': ['sqrt', 0.25, 0.5, 0.75, 1.0],
#    'min_samples_split': [2, 4, 6]
#}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_train_chi, y_train)

print(grid_search.best_params_)
print(grid_search.best_estimator_)
print(grid_search.best_score_)

Fitting 3 folds for each of 288 candidates, totalling 864 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    5.0s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   31.6s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 864 out of 864 | elapsed:  3.3min finished


{'bootstrap': True, 'max_depth': 90, 'max_features': 3, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 200}
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=90, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
0.9422


In [17]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=90, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=3, min_samples_split=8,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)


#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4244 49 245 462
pod:  0.6534653465346535
pof:  0.011413929652923364
AUC:  0.8210257084408651
accuracy:  0.9412


In [18]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
classifier =RandomForestClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


4233 60 273 434
pod:  0.6138613861386139
pof:  0.013976240391334731
AUC:  0.7999425728736396


In [19]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier(criterion='entropy', splitter='best')
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4050 243 222 485
pod:  0.685997171145686
pof:  0.05660377358490566
AUC:  0.8146966987803902
accuracy:  0.907


In [20]:
#Decision Tree
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4011 282 222 485
pod:  0.685997171145686
pof:  0.06568832983927324
AUC:  0.8101544206532064


In [21]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

#classifier = GradientBoostingClassifier(max_features=None, max_depth=3, criterion='friedman_mse')
classifier = GradientBoostingClassifier()
classifier.fit(x_train_chi, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = x_train_chi, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4242 51 241 466
pod:  0.6591230551626591
pof:  0.011879804332634521
AUC:  0.8236216254150123
accuracy:  0.9416


In [22]:
#Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()
classifier.fit(X_train, y_train) 
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
y_pred  = cross_val_predict(estimator = classifier, X = X_train, y = y_train, cv = 10)
tn, fp, fn, tp  = confusion_matrix(y_train, y_pred).ravel()
print(tn, fp, fn, tp)
pod=tp/(tp+fn)

print('pod: ',pod)
pof=fp/(fp+tn)
print ('pof: ',pof)
auc_val=(1+pod-pof)/2
print ('AUC: ',auc_val)

accuracy=(tp+tn)/(tp+fn+fp+tn)
print ('accuracy: ',accuracy)

#from sklearn.model_selection import cross_val_score
#roc_auc = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10,  scoring='roc_auc')
#print('roc_auc: ',roc_auc.mean())

#from sklearn.metrics import roc_curve, auc
#fpr, tpr, thresholds = roc_curve(y_train, y_pred)
#auc = auc(fpr, tpr)
#print('auc: ',auc)

4240 53 241 466
pod:  0.6591230551626591
pof:  0.012345679012345678
AUC:  0.8233886880751566
accuracy:  0.9412


In [23]:
df_x_train_chi = pd.DataFrame(x_train_chi)
df_x_train_chi.head()

#FNN
def get_FNN_Predict(X2_train, X2_test, y2_train, y2_test):
    from keras.models import Sequential
    from keras.layers import Dense
    #create model
    model = Sequential()
    #get number of columns in training data
    n_cols = X2_train.shape[1]
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(250, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    #compile model using mse as a measure of model performance
    #model.compile(optimizer='adam', loss='mean_squared_error')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  
    from keras.callbacks import EarlyStopping
    #set early stopping monitor so the model stops training when it won't improve anymore
    early_stopping_monitor = EarlyStopping(patience=3)
    #train model
    model.fit(X2_train, y2_train, validation_split=0.2, epochs=30, callbacks=[early_stopping_monitor])
    y2_pred = model.predict(X2_test)
    print('y2_pred: ',y2_pred)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_FNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

print('tn_list: ',tn_list)
print ('fp_list: ',fp_list)
print ('fn_list: ',fn_list)
print ('tp_list: ',tp_list)


tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 


Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Train on 3599 samples, validate on 900 samples
Epoch 1/30
3599/3599 [==============================] - 1s 150us/step - loss: 0.4433 - acc: 0.8586 - val_loss: 0.4237 - val_acc: 0.8589
Epoch 2/30
3599/3599 [==============================] - 0s 56us/step - loss: 0.4152 - acc: 0.8586 - val_loss: 0.4125 - val_acc: 0.8589
Epoch 3/30
3599/3599 [==============================] - 0s 56us/step - loss: 0.4071 - acc: 0.8586 - val_loss: 0.4018 - val_acc: 0.8589
Epoch 4/30
3599/3599 [==============================] - 0s 56us/step - loss: 0.3992 - acc: 0.8588 - val_loss: 0.3973 - val_acc: 0.8589
Epoch 5/30
3599/3599 [==============================] - 0s 60us/step - loss: 0.3888 - acc: 0.8597 - val_loss: 0.3894 - val_acc: 0.8611
Epoch 6/30
3599/3599 [==============================] - 0s 61us/step - loss: 0.3849 - acc: 0.8608 - val_loss: 0.3814 - val_acc: 0.8589
Epoch 7/30
3599/3599 [

Train on 3599 samples, validate on 900 samples
Epoch 1/30
3599/3599 [==============================] - 1s 154us/step - loss: 0.4703 - acc: 0.8519 - val_loss: 0.4154 - val_acc: 0.8589
Epoch 2/30
3599/3599 [==============================] - 0s 52us/step - loss: 0.4219 - acc: 0.8586 - val_loss: 0.4607 - val_acc: 0.8589
Epoch 3/30
3599/3599 [==============================] - 0s 60us/step - loss: 0.4105 - acc: 0.8586 - val_loss: 0.3962 - val_acc: 0.8600
Epoch 4/30
3599/3599 [==============================] - 0s 56us/step - loss: 0.3991 - acc: 0.8588 - val_loss: 0.3987 - val_acc: 0.8578
Epoch 5/30
3599/3599 [==============================] - 0s 58us/step - loss: 0.3947 - acc: 0.8580 - val_loss: 0.3799 - val_acc: 0.8611
Epoch 6/30
3599/3599 [==============================] - 0s 60us/step - loss: 0.3819 - acc: 0.8588 - val_loss: 0.4053 - val_acc: 0.8422
Epoch 7/30
3599/3599 [==============================] - 0s 58us/step - loss: 0.3703 - acc: 0.8605 - val_loss: 0.3630 - val_acc: 0.8633
Epoch 8

Train on 3599 samples, validate on 900 samples
Epoch 1/30
3599/3599 [==============================] - 1s 175us/step - loss: 0.4175 - acc: 0.8583 - val_loss: 0.4161 - val_acc: 0.8589
Epoch 2/30
3599/3599 [==============================] - 0s 54us/step - loss: 0.4021 - acc: 0.8586 - val_loss: 0.4073 - val_acc: 0.8600
Epoch 3/30
3599/3599 [==============================] - 0s 54us/step - loss: 0.3974 - acc: 0.8597 - val_loss: 0.3911 - val_acc: 0.8589
Epoch 4/30
3599/3599 [==============================] - 0s 58us/step - loss: 0.3933 - acc: 0.8597 - val_loss: 0.3897 - val_acc: 0.8589
Epoch 5/30
3599/3599 [==============================] - 0s 53us/step - loss: 0.3918 - acc: 0.8588 - val_loss: 0.3838 - val_acc: 0.8600
Epoch 6/30
3599/3599 [==============================] - 0s 54us/step - loss: 0.3875 - acc: 0.8586 - val_loss: 0.3803 - val_acc: 0.8600
Epoch 7/30
3599/3599 [==============================] - 0s 53us/step - loss: 0.3831 - acc: 0.8588 - val_loss: 0.3798 - val_acc: 0.8600
Epoch 8

y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [1.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

...

Train on 3600 samples, validate on 900 samples
Epoch 1/30
3600/3600 [==============================] - 1s 227us/step - loss: 0.4355 - acc: 0.8583 - val_loss: 0.4065 - val_acc: 0.8589
Epoch 2/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.4098 - acc: 0.8583 - val_loss: 0.4127 - val_acc: 0.8589
Epoch 3/30
3600/3600 [==============================] - 0s 57us/step - loss: 0.4007 - acc: 0.8567 - val_loss: 0.3982 - val_acc: 0.8578
Epoch 4/30
3600/3600 [==============================] - 0s 56us/step - loss: 0.3886 - acc: 0.8606 - val_loss: 0.3805 - val_acc: 0.8589
Epoch 5/30
3600/3600 [==============================] - 0s 63us/step - loss: 0.3712 - acc: 0.8639 - val_loss: 0.3763 - val_acc: 0.8556
Epoch 6/30
3600/3600 [==============================] - 0s 56us/step - loss: 0.3631 - acc: 0.8664 - val_loss: 0.3786 - val_acc: 0.8622
Epoch 7/30
3600/3600 [==============================] - 0s 56us/step - loss: 0.3526 - acc: 0.8692 - val_loss: 0.3650 - val_acc: 0.8589
Epoch 8

 [0.]]
pod 1st:  0.2676056338028169
Train on 3600 samples, validate on 900 samples
Epoch 1/30
3600/3600 [==============================] - 1s 244us/step - loss: 0.4459 - acc: 0.8519 - val_loss: 0.4203 - val_acc: 0.8589
Epoch 2/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.4174 - acc: 0.8586 - val_loss: 0.3943 - val_acc: 0.8589
Epoch 3/30
3600/3600 [==============================] - 0s 56us/step - loss: 0.4082 - acc: 0.8583 - val_loss: 0.3915 - val_acc: 0.8589
Epoch 4/30
3600/3600 [==============================] - 0s 56us/step - loss: 0.3961 - acc: 0.8594 - val_loss: 0.3856 - val_acc: 0.8567
Epoch 5/30
3600/3600 [==============================] - 0s 58us/step - loss: 0.3979 - acc: 0.8625 - val_loss: 0.3794 - val_acc: 0.8589
Epoch 6/30
3600/3600 [==============================] - 0s 63us/step - loss: 0.3874 - acc: 0.8625 - val_loss: 0.3858 - val_acc: 0.8567
Epoch 7/30
3600/3600 [==============================] - 0s 57us/step - loss: 0.3798 - acc: 0.8603 - val_lo

Train on 3600 samples, validate on 901 samples
Epoch 1/30
3600/3600 [==============================] - 1s 262us/step - loss: 0.4455 - acc: 0.8522 - val_loss: 0.4592 - val_acc: 0.8590
Epoch 2/30
3600/3600 [==============================] - 0s 60us/step - loss: 0.4270 - acc: 0.8583 - val_loss: 0.4076 - val_acc: 0.8590
Epoch 3/30
3600/3600 [==============================] - 0s 59us/step - loss: 0.4112 - acc: 0.8583 - val_loss: 0.3990 - val_acc: 0.8590
Epoch 4/30
3600/3600 [==============================] - 0s 66us/step - loss: 0.4000 - acc: 0.8583 - val_loss: 0.3986 - val_acc: 0.8590
Epoch 5/30
3600/3600 [==============================] - 0s 66us/step - loss: 0.3962 - acc: 0.8575 - val_loss: 0.3896 - val_acc: 0.8590
Epoch 6/30
3600/3600 [==============================] - 0s 65us/step - loss: 0.3819 - acc: 0.8586 - val_loss: 0.3933 - val_acc: 0.8590
Epoch 7/30
3600/3600 [==============================] - 0s 64us/step - loss: 0.3720 - acc: 0.8586 - val_loss: 0.3931 - val_acc: 0.8568
Epoch 8

Train on 3600 samples, validate on 901 samples
Epoch 1/30
3600/3600 [==============================] - 1s 282us/step - loss: 0.4395 - acc: 0.8503 - val_loss: 0.4069 - val_acc: 0.8646
Epoch 2/30
3600/3600 [==============================] - 0s 61us/step - loss: 0.4235 - acc: 0.8569 - val_loss: 0.3945 - val_acc: 0.8646
Epoch 3/30
3600/3600 [==============================] - 0s 60us/step - loss: 0.4169 - acc: 0.8569 - val_loss: 0.4265 - val_acc: 0.8646
Epoch 4/30
3600/3600 [==============================] - 0s 66us/step - loss: 0.4103 - acc: 0.8569 - val_loss: 0.4001 - val_acc: 0.8646
Epoch 5/30
3600/3600 [==============================] - 0s 63us/step - loss: 0.4041 - acc: 0.8572 - val_loss: 0.3959 - val_acc: 0.8646
y2_pred:  [[0.03956422]
 [0.05609426]
 [0.04254943]
 [0.05704406]
 [0.05415663]
 [0.167662  ]
 [0.20222375]
 [0.05114624]
 [0.11352339]
 [0.094354  ]
 [0.06275967]
 [0.07305896]
 [0.11421227]
 [0.10956627]
 [0.09015498]
 [0.14003941]
 [0.10116643]
 [0.1566388 ]
 [0.15700802]
 

Train on 3600 samples, validate on 901 samples
Epoch 1/30
3600/3600 [==============================] - 1s 300us/step - loss: 0.4425 - acc: 0.8550 - val_loss: 0.4009 - val_acc: 0.8646
Epoch 2/30
3600/3600 [==============================] - 0s 69us/step - loss: 0.4162 - acc: 0.8575 - val_loss: 0.3902 - val_acc: 0.8646
Epoch 3/30
3600/3600 [==============================] - 0s 65us/step - loss: 0.4066 - acc: 0.8564 - val_loss: 0.3908 - val_acc: 0.8635
Epoch 4/30
3600/3600 [==============================] - 0s 63us/step - loss: 0.3913 - acc: 0.8567 - val_loss: 0.3784 - val_acc: 0.8657
Epoch 5/30
3600/3600 [==============================] - 0s 63us/step - loss: 0.3861 - acc: 0.8558 - val_loss: 0.3871 - val_acc: 0.8590
Epoch 6/30
3600/3600 [==============================] - 0s 70us/step - loss: 0.3801 - acc: 0.8628 - val_loss: 0.3805 - val_acc: 0.8624
Epoch 7/30
3600/3600 [==============================] - 0s 81us/step - loss: 0.3724 - acc: 0.8603 - val_loss: 0.3653 - val_acc: 0.8602
Epoch 8

In [24]:
#RNN
def get_RNN_Predict(X2_train, X2_test, y2_train, y2_test):
    import pandas as pd
    from keras.models import Sequential
    from keras.layers import Dense,Dropout, LSTM, GRU
    from keras.layers import Embedding
    max_features = 10000 # number of words to consider as features
    import numpy as np
    #create model 
    model = Sequential()
    model.add(Embedding(max_features, 32))
    model.add(LSTM(32))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='rmsprop',loss='binary_crossentropy', metrics=['acc'])
    history = model.fit(X2_train, y2_train, epochs=5, batch_size=128, validation_split=0.2)
    y2_pred = model.predict(X2_test)
    y22_pred=y2_pred.round()
    print('y22_pred: ',y22_pred)
    return y22_pred

pod_list = []
pof_list = []
auc_val_list = []
tn_list= []
fp_list= []
fn_list= []
tp_list= []
    
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=10)
for train_index, test_index in folds.split(df_x_train_chi,y_train):
    X2_train, X2_test, y2_train, y2_test=df_x_train_chi.iloc[train_index], df_x_train_chi.iloc[test_index],y_train.iloc[train_index], y_train.iloc[test_index]
    y_pred = get_RNN_Predict(X2_train, X2_test, y2_train, y2_test)
    tn, fp, fn, tp  = confusion_matrix(y2_test, y_pred).ravel()
    tn_list.append(tn)
    fp_list.append(fp)
    fn_list.append(fn)
    tp_list.append(tp)
    
    pod=tp/(tp+fn)
    print('pod 1st: ',pod)
    pof=fp/(fp+tn)
    auc_val=(1+pod-pof)/2
    #break
    pod_list.append(pod)
    pof_list.append(pof)
    auc_val_list.append(auc_val)

print('pod: ',pod_list)
print ('pof: ',pof_list)
print ('auc: ',auc_val_list)

tn=sum(tn_list)
fp=sum(fp_list) 
fn=sum(fn_list) 
tp=sum(tp_list)
print(tn, fp, fn, tp)
 

Train on 3599 samples, validate on 900 samples
Epoch 1/5
3599/3599 [==============================] - 2s 498us/step - loss: 0.4966 - acc: 0.8450 - val_loss: 0.4086 - val_acc: 0.8589
Epoch 2/5
3599/3599 [==============================] - 0s 88us/step - loss: 0.3948 - acc: 0.8586 - val_loss: 0.4030 - val_acc: 0.8589
Epoch 3/5
3599/3599 [==============================] - 0s 81us/step - loss: 0.3835 - acc: 0.8586 - val_loss: 0.3715 - val_acc: 0.8589
Epoch 4/5
3599/3599 [==============================] - 0s 81us/step - loss: 0.3762 - acc: 0.8586 - val_loss: 0.3850 - val_acc: 0.8589
Epoch 5/5
3599/3599 [==============================] - 0s 86us/step - loss: 0.3719 - acc: 0.8586 - val_loss: 0.3756 - val_acc: 0.8600
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]

Train on 3600 samples, validate on 900 samples
Epoch 1/5
3600/3600 [==============================] - 2s 642us/step - loss: 0.4979 - acc: 0.8486 - val_loss: 0.4174 - val_acc: 0.8589
Epoch 2/5
3600/3600 [==============================] - 0s 85us/step - loss: 0.3977 - acc: 0.8586 - val_loss: 0.3876 - val_acc: 0.8589
Epoch 3/5
3600/3600 [==============================] - 0s 87us/step - loss: 0.3882 - acc: 0.8586 - val_loss: 0.4044 - val_acc: 0.8589
Epoch 4/5
3600/3600 [==============================] - 0s 82us/step - loss: 0.3856 - acc: 0.8586 - val_loss: 0.3794 - val_acc: 0.8589
Epoch 5/5
3600/3600 [==============================] - 0s 86us/step - loss: 0.3788 - acc: 0.8586 - val_loss: 0.3640 - val_acc: 0.8589
y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 

y22_pred:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]